In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Users interact with posts (e.g., 1 = liked, 0 = no interaction)
data = {
    'Post1': [1, 0, 1, 0, 0],
    'Post2': [1, 1, 0, 0, 0],
    'Post3': [0, 1, 1, 1, 0],
    'Post4': [0, 0, 0, 1, 1],
    'Post5': [0, 0, 1, 1, 1],
}

users = ['UserA', 'UserB', 'UserC', 'UserD', 'UserE']

df = pd.DataFrame(data, index=users)
print("User-Post Matrix:\n", df)


User-Post Matrix:
        Post1  Post2  Post3  Post4  Post5
UserA      1      1      0      0      0
UserB      0      1      1      0      0
UserC      1      0      1      0      1
UserD      0      0      1      1      1
UserE      0      0      0      1      1


In [2]:
df.head()

,Post1,Post2,Post3,Post4,Post5
UserA,1,1,0,0,0
UserB,0,1,1,0,0
UserC,1,0,1,0,1
UserD,0,0,1,1,1
UserE,0,0,0,1,1


# user-user recommendation, user_based

In [3]:
# Compute cosine similarity between users
similarity_matrix = cosine_similarity(df)
similar_users = pd.DataFrame(similarity_matrix, index=users, columns=users)
print("\nUser Similarity Matrix:\n", similar_users)


User Similarity Matrix:
           UserA     UserB     UserC     UserD     UserE
UserA  1.000000  0.500000  0.408248  0.000000  0.000000
UserB  0.500000  1.000000  0.408248  0.408248  0.000000
UserC  0.408248  0.408248  1.000000  0.666667  0.408248
UserD  0.000000  0.408248  0.666667  1.000000  0.816497
UserE  0.000000  0.000000  0.408248  0.816497  1.000000


In [4]:
target_user = 'UserA'
target_index = users.index(target_user)

# Get similarity scores for UserA
user_similarities = similarity_matrix[target_index]

# Weighted sum of other users' interactions
scores = np.dot(user_similarities, df.values)

# Avoid recommending items the user already interacted with
user_seen = df.loc[target_user].values
recommendation_scores = scores * (user_seen == 0)

# Rank and show top recommendations
post_names = df.columns
recommendations = pd.Series(recommendation_scores, index=post_names)
top_recommendations = recommendations.sort_values(ascending=False)

print(f"\nTop Recommendations for {target_user}:\n", top_recommendations)



Top Recommendations for UserA:
 Post3    0.908248
Post5    0.408248
Post1    0.000000
Post2    0.000000
Post4    0.000000
dtype: float64


# item-item recommendation, item_based

In [6]:
item_user_matrix = df.T
print("\nItem-User Matrix:\n", item_user_matrix)


Item-User Matrix:
        UserA  UserB  UserC  UserD  UserE
Post1      1      0      1      0      0
Post2      1      1      0      0      0
Post3      0      1      1      1      0
Post4      0      0      0      1      1
Post5      0      0      1      1      1


In [7]:
# Cosine similarity between items (posts)
item_sim = cosine_similarity(item_user_matrix)
item_sim_df = pd.DataFrame(item_sim, index=df.columns, columns=df.columns)
print("\nItem (Post) Similarity Matrix:\n", item_sim_df)



Item (Post) Similarity Matrix:
           Post1     Post2     Post3     Post4     Post5
Post1  1.000000  0.500000  0.408248  0.000000  0.408248
Post2  0.500000  1.000000  0.408248  0.000000  0.000000
Post3  0.408248  0.408248  1.000000  0.408248  0.666667
Post4  0.000000  0.000000  0.408248  1.000000  0.816497
Post5  0.408248  0.000000  0.666667  0.816497  1.000000


In [12]:
target_user = 'UserA'
user_ratings = df.loc[target_user]
# print(user_ratings)
# Score = similarity of an item to others * user's interactions with those others
scores = item_sim_df.dot(user_ratings)
# print(scores)
# Don't recommend items the user already interacted with
already_seen = user_ratings == 1
scores[already_seen] = 0

# Show recommendations
recommendations = scores.sort_values(ascending=False)
print(f"\nItem-Based Recommendations for {target_user}:\n", recommendations)



Item-Based Recommendations for UserA:
 Post3    0.816497
Post5    0.408248
Post1    0.000000
Post2    0.000000
Post4    0.000000
dtype: float64


# MF

In [1]:
import numpy as np

# User-item interaction matrix (rows = users, columns = posts)
# 0 means no interaction, 1 means interaction
R = np.array([
    [5, 3, 0, 1, 0],
    [4, 0, 0, 1, 2],
    [1, 1, 0, 5, 0],
    [0, 0, 5, 4, 0],
    [0, 1, 5, 4, 0],
], dtype=float)

print("Original User-Item Matrix:\n", R)

Original User-Item Matrix:
 [[5. 3. 0. 1. 0.]
 [4. 0. 0. 1. 2.]
 [1. 1. 0. 5. 0.]
 [0. 0. 5. 4. 0.]
 [0. 1. 5. 4. 0.]]


In [2]:
from numpy.linalg import svd

# Perform SVD
U, sigma, Vt = svd(R, full_matrices=False)

print("U shape:", U.shape)
print("sigma shape:", sigma.shape)
print("Vt shape:", Vt.shape)

U shape: (5, 5)
sigma shape: (5,)
Vt shape: (5, 5)


In [3]:
k = 2  # number of latent features to keep

# Create diagonal matrix for sigma
S = np.diag(sigma[:k])

# Reduce U and Vt to k features
U_reduced = U[:, :k]
Vt_reduced = Vt[:k, :]

print("U_reduced shape:", U_reduced.shape)
print("S shape:", S.shape)
print("Vt_reduced shape:", Vt_reduced.shape)


U_reduced shape: (5, 2)
S shape: (2, 2)
Vt_reduced shape: (2, 5)


In [4]:
# Approximate original matrix
R_approx = np.dot(np.dot(U_reduced, S), Vt_reduced)
print("Approximated User-Item Matrix:\n", R_approx)


Approximated User-Item Matrix:
 [[ 4.97610591  1.99126782 -0.53812927  1.6547332   0.86865619]
 [ 3.56653408  1.42685597 -0.38836997  1.1832309   0.62261781]
 [ 1.83307084  1.03235209  2.09065716  2.97862929  0.29931595]
 [-0.36542033  0.43169409  4.46627415  4.46030584 -0.1037754 ]
 [-0.06527349  0.5545843   4.45512256  4.58216885 -0.05157259]]


In [5]:
user_id = 0
user_interactions = R[user_id]
user_predictions = R_approx[user_id]

# Recommend posts with zero original interaction but high predicted score
recommendations = [(i, score) for i, (orig, score) in enumerate(zip(user_interactions, user_predictions)) if orig == 0]
recommendations.sort(key=lambda x: x[1], reverse=True)

print(f"Recommended posts for user {user_id}:")
for post_id, score in recommendations:
    print(f"Post {post_id} with predicted rating {score:.2f}")


Recommended posts for user 0:
Post 4 with predicted rating 0.87
Post 2 with predicted rating -0.54
